# Speed Distribution

$$ \huge p(v) = \frac{v}{\sigma^2} e^{-\frac{v^2}{2\sigma^2}} $$

In [ ]:
print("\n Global speed distribution analysis")
blueTrajs, redTraj = get_trajs(nDrops, red_particle_idx, rawTrajs)
blueTrajs_smooth, redTraj_smooth = get_trajs(nDrops, red_particle_idx, smoothTrajs) 

bin_borders = np.arange(0, 100, .2)
bin_centers = np.arange(0, 100, .2)[:-1] + .2 / 2
x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 10000)

v_blue = ys.speed_ensemble(blueTrajs, step = 10)
v_red = ys.speed_ensemble(redTraj, step = 10)
T_blue, T_blue_std = fit_hist(v_blue, bin_borders, MB_2D, [1.])
T_red, T_red_std = fit_hist(v_red, bin_borders, MB_2D, [1.])
print("Raw Trajectories")
print(f"Blue Particles σ: {T_blue[0]} ± {T_blue_std[0]}")
print(f"Red Particle σ: {T_red[0]} ± {T_red_std[0]}")

v_blue_smooth = ys.speed_ensemble(blueTrajs_smooth, step = 10)
v_red_smooth = ys.speed_ensemble(redTraj_smooth, step = 10)
T_blue_smooth, T_blue_smooth_std = fit_hist(v_blue_smooth, bin_borders, MB_2D, [1.])
T_red_smooth, T_red_smooth_std = fit_hist(v_red_smooth, bin_borders, MB_2D, [1.])
print("Smooth Trajectories")
print(f"Blue Particles σ: {T_blue_smooth[0]} ± {T_blue_smooth_std[0]}")
print(f"Red Particle σ: {T_red_smooth[0]} ± {T_red_smooth_std[0]}")

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 5))
ax.hist(v_blue, bins = bin_borders, **default_kwargs_blue)
ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_blue), 'k-', label = f"$T = {T_blue[0]:.2f} \pm {T_blue_std[0]:.2f}$")
ax.set(title = "Blue particles velocity distribution", xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
ax.legend()

ax1.hist(v_red, bins = bin_borders, **default_kwargs_red)
ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_red), 'k-', label = f"$T = {T_red[0]:.2f} \pm {T_red_std[0]:.2f}$")
ax1.set(title = "Red particle velocity distribution", xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
ax1.legend()

plt.suptitle("Raw trajectories")
plt.tight_layout()
if save_verb: plt.savefig("./results/speed_distribution/speed_raw.png", bbox_inches='tight')
if show_verb: 
    plt.show()
else:
    plt.close()

fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 5))
ax.hist(v_blue_smooth, bins = bin_borders, **default_kwargs_blue)
ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_blue_smooth), 'k-', 
            label = f"$T = {T_blue_smooth[0]:.2f} \pm {T_blue_smooth_std[0]:.2f}$")
ax.set(title = "Blue particles velocity distribution", xlabel = f"speed [{units}]", 
            ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
ax.legend()

ax1.hist(v_red_smooth, bins = bin_borders, **default_kwargs_red)
ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, T_red_smooth), 'k-', 
            label = f"$T = {T_red_smooth[0]:.2f} \pm {T_red_smooth_std[0]:.2f}$")
ax1.set(title = "Red particle velocity distribution", xlabel = f"speed [{units}]",
            ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
ax1.legend()
plt.suptitle("Smooth trajectories")
plt.tight_layout()
if save_verb: plt.savefig("./results/speed_distribution/speed_smooth.png", bbox_inches='tight')
if show_verb: 
    plt.show()
else:
    plt.close()

## Windowed Analysis

In [ ]:
print("\n Windowed speed distribution Analysis")
v_blue_wind, v_red_wind = speed_windowed(nDrops, nSteps, startFrames, endFrames, red_particle_idx, rawTrajs)
blue_fit_wind = np.ones((nSteps, 2))
red_fit_wind = np.ones((nSteps, 2))

for k in range(nSteps):
    blue_fit_wind[k, 0], blue_fit_wind[k, 1] = fit_hist(v_blue_wind[k], bin_borders, MB_2D, [1.])
    red_fit_wind[k, 0], red_fit_wind[k, 1] = fit_hist(v_red_wind[k], bin_borders, MB_2D, [1.])

v_blue_wind_smooth, v_red_wind_smooth = speed_windowed(nDrops, nSteps, startFrames, endFrames, red_particle_idx, smoothTrajs)
blue_fit_wind_smooth = np.ones((nSteps, 2))
red_fit_wind_smooth = np.ones((nSteps, 2))

for k in range(nSteps):
    blue_fit_wind_smooth[k, 0], blue_fit_wind_smooth[k, 1] = fit_hist(v_blue_wind_smooth[k], bin_borders, MB_2D, [1.])
    red_fit_wind_smooth[k, 0], red_fit_wind_smooth[k, 1] = fit_hist(v_red_wind_smooth[k], bin_borders, MB_2D, [1.])

### PLOT

In [ ]:
if animated_plot_verb:
    fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 5))
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True


    def prepare_animation(bar_container, bar_container2):
        def animate(frame):
            # update titles
            title.set_text(f"Raw Trajectories - window {startFrames[frame]/10} - {endFrames[frame]/10} seconds")
            #title2.set_text(f"Red particle velocity pdf {startFrames[frame]/10} - {endFrames[frame]/10} seconds")

            # update histogram 1
            n, _ = np.histogram(v_blue_wind[frame], bin_borders, density = True)
            for count, rect in zip(n, bar_container.patches):
                rect.set_height(count)
            
            line.set_ydata(MB_2D(x_interval_for_fit, blue_fit_wind[frame, 0]))

            # update histogram 2
            n2, _ = np.histogram(v_red_wind[frame], bin_borders, density = True)
            for count2, rect2 in zip(n2, bar_container2.patches):
                rect2.set_height(count2)
                
            line1.set_ydata(MB_2D(x_interval_for_fit, red_fit_wind[frame, 0]))

            return bar_container.patches, bar_container2.patches
        return animate

    _, _, bar_container = ax.hist(v_blue_wind[0], bin_borders, **default_kwargs_blue, label="blue particles")
    title = ax.set_title(f"Raw Trajectories - window {startFrames[0]/10} - {endFrames[0]/10} seconds")
    line, = ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, blue_fit_wind[0, 0]), label='fit')
    ax.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
    ax.legend()

    _, _, bar_container2 = ax1.hist(v_red_wind[0], bin_borders,  **default_kwargs_red, label="red particle")
    #title2 = ax1.set_title(f"Red particle velocity pdf {startFrames[0]/10} - {endFrames[0]/10} seconds")
    line1, = ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, red_fit_wind[0, 0]), label='fit')
    ax1.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
    ax1.legend()

    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container, bar_container2), nSteps, repeat=True, blit=False)
    if save_verb: ani.save('./results/speed_distribution/speed_raw_wind.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if show_verb: 
        plt.show()
    else:
        plt.close()

In [ ]:
if animated_plot_verb:
    fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 5))
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True


    def prepare_animation(bar_container, bar_container2):
        def animate(frame):
            # update titles
            title.set_text(f"Smooth Trajectories - window {startFrames[frame]/10} - {endFrames[frame]/10} seconds")
            #title2.set_text(f"Red particle velocity pdf {startFrames[frame]/10} - {endFrames[frame]/10} seconds")

            # update histogram 1
            n, _ = np.histogram(v_blue_wind_smooth[frame], bin_borders, density = True)
            for count, rect in zip(n, bar_container.patches):
                rect.set_height(count)
            
            line.set_ydata(MB_2D(x_interval_for_fit, blue_fit_wind_smooth[frame, 0]))

            # update histogram 2
            n2, _ = np.histogram(v_red_wind_smooth[frame], bin_borders, density = True)
            for count2, rect2 in zip(n2, bar_container2.patches):
                rect2.set_height(count2)
                
            line1.set_ydata(MB_2D(x_interval_for_fit, red_fit_wind_smooth[frame, 0]))

            return bar_container.patches, bar_container2.patches
        return animate

    _, _, bar_container = ax.hist(v_blue_wind_smooth[0], bin_borders, **default_kwargs_blue, label="blue particles")
    title = ax.set_title(f"Raw Trajectories - window {startFrames[0]/10} - {endFrames[0]/10} seconds")
    line, = ax.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, blue_fit_wind_smooth[0, 0]), label='fit')
    ax.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
    ax.legend()

    _, _, bar_container2 = ax1.hist(v_red_wind_smooth[0], bin_borders,  **default_kwargs_red, label="red particle")
    #title2 = ax1.set_title(f"Red particle velocity pdf {startFrames[0]/10} - {endFrames[0]/10} seconds")
    line1, = ax1.plot(x_interval_for_fit, MB_2D(x_interval_for_fit, red_fit_wind_smooth[0, 0]), label='fit')
    ax1.set(xlabel = f"speed [{units}]", ylabel = "pdf", xlim = (0, 30), ylim = (0, 0.3))
    ax1.legend()

    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container, bar_container2), nSteps, repeat=True, blit=False)
    if save_verb: ani.save('./results/speed_distribution/speed_smooth_wind.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if show_verb: 
        plt.show()
    else:
        plt.close()

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 4), sharex=True)
ax.errorbar(startFrames/10, blue_fit_wind[:, 0], yerr = blue_fit_wind[:, 1], fmt = 'b', label="blue particles")
ax.set(ylabel = "T [??]", ylim = (1, 5), title = "Effective Temperature - Raw Trajectories")
ax.legend()
ax.grid()
ax1.errorbar(startFrames/10, red_fit_wind[:, 0], yerr = red_fit_wind[:, 1], fmt = 'r', label="red particles")
ax1.set(xlabel ="time [s]", ylabel = "T [??]")
ax1.legend()
ax1.grid()
plt.tight_layout()
if save_verb: plt.savefig('./results/speed_distribution/T_eff_raw.png',  bbox_inches='tight')
if show_verb:
    plt.show()
else:
    plt.close()

    
fig, (ax, ax1) = plt.subplots(2, 1, figsize = (8, 4), sharex=True)
ax.errorbar(startFrames/10, blue_fit_wind_smooth[:, 0], yerr = blue_fit_wind_smooth[:, 1], fmt = 'b', label="blue particles")
ax.set(ylabel = "T [??]", ylim = (1, 5), title = "Effective Temperature - Smooth Trajectories")
ax.legend()
ax.grid()

ax1.errorbar(startFrames/10, red_fit_wind_smooth[:, 0], yerr = red_fit_wind_smooth[:, 1], fmt = 'r', label="red particles")
ax1.set(xlabel = "time [s]", ylabel = "T [??]")
ax1.legend()
ax1.grid()
plt.tight_layout()
if save_verb: plt.savefig('./results/speed_distribution/T_eff_smooth.png',  bbox_inches='tight')
if show_verb:
    plt.show()
else:
    plt.close()

# Turning Angles Distribution

$$ \huge f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}} $$

In [ ]:
print("\n Global turning angles analysis")
blueTrajs, redTraj = get_trajs(nDrops, red_particle_idx, rawTrajs)
blueTrajs_smooth, redTraj_smooth = get_trajs(nDrops, red_particle_idx, smoothTrajs)

bin_borders_turn = np.arange(-np.pi, np.pi + 0.0001, np.pi/50)
bin_centers_turn = bin_borders_turn[:-1] + np.diff(bin_borders_turn) / 2
x_interval_for_fit_turn = np.linspace(bin_borders_turn[0], bin_borders_turn[-1], 10000)

theta_blue = ys.turning_angles_ensemble(blueTrajs, centered = True)
theta_red = ys.turning_angles_ensemble(redTraj, centered = True)
# normal distribution fit
T_blue_rot, T_blue_rot_std = fit_hist(theta_blue, bin_borders_turn, normal_distr, [1., 0.])
T_red_rot, T_red_rot_std = fit_hist(theta_red, bin_borders_turn, normal_distr, [1., 0.])
print("Raw Trajectories")
print(f"Blue Particles σ: {T_blue_rot[0]} ± {T_blue_rot_std[0]}, μ: {T_blue_rot[1]} ± {T_blue_rot_std[1]}")
print(f"Red Particle σ: {T_red_rot[0]} ± {T_red_rot_std[0]}, μ: {T_red_rot[1]} ± {T_red_rot_std[1]}")


theta_blue_smooth = ys.turning_angles_ensemble(blueTrajs_smooth, centered = True)
theta_red_smooth = ys.turning_angles_ensemble(redTraj_smooth, centered = True)
# normal distribution fit
T_blue_rot_smooth, T_blue_rot_smooth_std = fit_hist(theta_blue_smooth, bin_borders_turn, normal_distr, [1., 0.])
T_red_rot_smooth, T_red_rot_smooth_std = fit_hist(theta_red_smooth, bin_borders_turn, normal_distr, [1., 0.])
print("Smooth Trajectories")
print(f"Blue Particles σ: {T_blue_rot_smooth[0]} ± {T_blue_rot_smooth_std[0]}, μ: {T_blue_rot_smooth[1]} ± {T_blue_rot_smooth_std[1]}")
print(f"Red Particle σ: {T_red_rot_smooth[0]} ± {T_red_rot_smooth_std[0]}, μ: {T_red_rot_smooth[1]} ± {T_red_rot_smooth_std[1]}")

In [ ]:
fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
ax.hist(theta_blue, bin_borders_turn, **default_kwargs_blue, label="blue particles")
ax.plot(x_interval_for_fit_turn, normal_distr(x_interval_for_fit_turn, *T_blue_rot), 'k-',
                label = f"$T = {T_blue_rot[0]:.2f} \pm {T_blue_rot_std[0]:.2f}$")
ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],[r'-$\pi$', r'$-\frac{\pi}{2}$', '$0$', r'$\frac{\pi}{2}$', r'$\pi$'])
ax.legend()
ax.set_ylim(0,0.5)

ax1.hist(theta_red, bin_borders_turn, **default_kwargs_red, label="red particle")
ax1.plot(x_interval_for_fit_turn, normal_distr(x_interval_for_fit_turn, *T_red_rot), 'k-',
                label = f"$T = {T_red_rot[0]:.2f} \pm {T_red_rot_std[0]:.2f}$")
ax1.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],[r'-$\pi$', r'$-\frac{\pi}{2}$', '$0$', r'$\frac{\pi}{2}$', r'$\pi$'])
ax1.legend()
ax1.set_ylim(0,0.5)
plt.suptitle("Turning angles pdf - Raw trajectories")
if save_verb: plt.savefig(f"./results/turning_angles/turn_ang_raw.png", bbox_inches = 'tight')
if show_verb:
    plt.show()
else:
    plt.close()

fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 4))
ax.hist(theta_blue_smooth, bin_borders_turn, **default_kwargs_blue, label="blue particles")
ax.plot(x_interval_for_fit_turn, normal_distr(x_interval_for_fit_turn,* T_blue_rot_smooth), 'k-',
                label = f"$T = {T_blue_rot_smooth[0]:.3f} \pm {T_blue_rot_smooth_std[0]:.3f}$")
ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],[r'-$\pi$', r'$-\frac{\pi}{2}$', '$0$', r'$\frac{\pi}{2}$', r'$\pi$'])
ax.legend(prop={'size': 8})

ax1.hist(theta_red_smooth, bin_borders_turn, **default_kwargs_red, label="red particle")
ax1.plot(x_interval_for_fit_turn, normal_distr(x_interval_for_fit_turn, *T_red_rot_smooth), 'k-',
                label = f"$T = {T_red_rot_smooth[0]:.3f} \pm {T_red_rot_smooth_std[0]:.3f}$")
ax1.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi],[r'-$\pi$', r'$-\frac{\pi}{2}$', '$0$', r'$\frac{\pi}{2}$', r'$\pi$'])
ax1.legend(prop={'size': 8})

plt.suptitle("Turning angles pdf - Smooth trajectories")
if save_verb: plt.savefig(f"./results/turning_angles/turn_ang_smooth.png", bbox_inches = 'tight')
if show_verb:
    plt.show()
else:
    plt.close()

## Windowed Analysis

In [ ]:
print("\n Windowed turning angles analysis")
theta_blue_wind, theta_red_wind = theta_windowed(nDrops, nSteps, startFrames, endFrames, red_particle_idx, rawTrajs)
theta_blue_wind_smooth, theta_red_wind_smooth = theta_windowed(nDrops, nSteps, startFrames, endFrames, red_particle_idx, smoothTrajs)

blue_fit_wind_turn = np.ones((nSteps, 2, 2))
red_fit_wind_turn = np.ones((nSteps, 2, 2))

for k in range(nSteps):
    blue_fit_wind_turn[k, 0], blue_fit_wind_turn[k, 1] = fit_hist(theta_blue_wind[k], bin_borders_turn, normal_distr, [1., 0.])
    red_fit_wind_turn[k, 0], red_fit_wind_turn[k, 1] = fit_hist(theta_red_wind[k], bin_borders_turn, normal_distr, [1., 0.])

blue_fit_wind_turn_smooth = np.ones((nSteps, 2, 2))
red_fit_wind_turn_smooth = np.ones((nSteps, 2, 2))

for k in range(nSteps):
    blue_fit_wind_turn_smooth[k, 0], blue_fit_wind_turn_smooth[k, 1] = fit_hist(theta_blue_wind_smooth[k], bin_borders_turn, normal_distr, [1., 0.])
    red_fit_wind_turn_smooth[k, 0], red_fit_wind_turn_smooth[k, 1] = fit_hist(theta_red_wind_smooth[k], bin_borders_turn, normal_distr, [1., 0.])

### PLOT

In [ ]:
if animated_plot_verb:
    fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 5))
    anim_running = True

    def onClick(event):
        global anim_running
        if anim_running:
            ani.event_source.stop()
            anim_running = False
        else:
            ani.event_source.start()
            anim_running = True

    def prepare_animation(bar_container, bar_container2):
        def animate(frame):
            title.set_text(f"Turning angles pdf - Smooth Trajectories - window {startFrames[frame]/10} - {endFrames[frame]/10} seconds")
            n, _ = np.histogram(theta_blue_wind_smooth[frame], bin_borders_turn, density = True)
            for count, rect in zip(n, bar_container.patches):
                rect.set_height(count)
            line.set_ydata(normal_distr(x_interval_for_fit_turn, *blue_fit_wind_turn_smooth[frame, 0]))
                
            n2, _ = np.histogram(theta_red_wind_smooth[frame], bin_borders_turn, density = True)
            for count2, rect2 in zip(n2, bar_container2.patches):
                rect2.set_height(count2)
            
            line1.set_ydata(normal_distr(x_interval_for_fit_turn, *red_fit_wind_turn_smooth[frame, 0]))
                
            return bar_container.patches, bar_container2.patches
        return animate

    _, _, bar_container = ax.hist(theta_red_wind_smooth[0], bin_borders_turn, **default_kwargs_blue, label="blue particles")
    line, = ax.plot(x_interval_for_fit_turn, normal_distr(x_interval_for_fit_turn, *blue_fit_wind_turn_smooth[0, 0]), label='fit')
    title = ax.set_title(f"Turning angles pdf - Smooth Trajectories - window {startFrames[0]/10} - {endFrames[0]/10} seconds")

    ax.set(ylabel = "pdf", ylim = (0, 3))
    ax.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi], [r'-$\pi$', r'$-\frac{\pi}{2}$', '$0$', r'$\frac{\pi}{2}$', r'$\pi$'])

    _, _, bar_container2 = ax1.hist(theta_red_wind_smooth[0], bin_borders_turn,  **default_kwargs_red, label="red particle")
    line1, = ax1.plot(x_interval_for_fit_turn, normal_distr(x_interval_for_fit_turn, *red_fit_wind_turn_smooth[0, 0]), label='fit')
    ax1.set(ylabel = "pdf", ylim = (0, 3))
    ax1.set_xticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi], [r'-$\pi$', r'$-\frac{\pi}{2}$', '$0$', r'$\frac{\pi}{2}$', r'$\pi$'])

    plt.tight_layout()
    fig.canvas.mpl_connect('button_press_event', onClick)
    ani = matplotlib.animation.FuncAnimation(fig, prepare_animation(bar_container, bar_container2), nSteps, repeat=True, blit=False)
    ani.save('./results/turning_angles/turn_ang_wind_smooth.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
    if anim_show_verb:
        plt.show()
    else:
        plt.close()

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 4), sharex=True)
ax.plot(startFrames/10, blue_fit_wind_turn[:, 0, 0], 'b', label="blue particles")
ax.fill_between(startFrames/10, blue_fit_wind_turn[:, 0, 0] - blue_fit_wind_turn[:, 1, 0],
                 blue_fit_wind_turn[:, 0, 0] + blue_fit_wind_turn[:, 1, 0], color='b', alpha=0.2)
ax.set(ylabel = "T [??]", title = "Effective Temperature - Raw Trajectories")
ax.legend()
ax.grid()

ax1.plot(startFrames/10, red_fit_wind_turn[:, 0, 0], 'r', label="red particles")
ax1.fill_between(startFrames/10, red_fit_wind_turn[:, 0, 0] - red_fit_wind_turn[:, 1, 0],
                    red_fit_wind_turn[:, 0, 0] + red_fit_wind_turn[:, 1, 0], color='r', alpha=0.2)
ax1.set(xlabel ="time [s]", ylabel = "T [??]")
ax1.legend()
ax1.grid()

plt.tight_layout()
if save_verb: plt.savefig('./results/turning_angles/effective_T_raw.png',  bbox_inches='tight')
if show_verb:
    plt.show()
else:
    plt.close()

fig, (ax, ax1) = plt.subplots(2, 1, figsize=(8, 4), sharex=True)
ax.plot(startFrames/10, blue_fit_wind_turn_smooth[:, 0, 0], 'b', label="blue particles")
ax.fill_between(startFrames/10, blue_fit_wind_turn_smooth[:, 0, 0] - blue_fit_wind_turn_smooth[:, 1, 0],
                    blue_fit_wind_turn_smooth[:, 0, 0] + blue_fit_wind_turn_smooth[:, 1, 0], color='b', alpha=0.2)
ax.set(ylabel = "T [??]", title = "Rotational effective temperature - Smooth Trajectories")
ax.legend()
ax.grid()

ax1.plot(startFrames/10, red_fit_wind_turn_smooth[:, 0, 0], 'r', label="red particles")
ax1.fill_between(startFrames/10, red_fit_wind_turn_smooth[:, 0, 0] - red_fit_wind_turn_smooth[:, 1, 0],
                        red_fit_wind_turn_smooth[:, 0, 0] + red_fit_wind_turn_smooth[:, 1, 0], color='r', alpha=0.2)
ax1.set(xlabel = "time [s]", ylabel = "T [??]")
ax1.legend()
ax1.grid()

plt.tight_layout()
if save_verb: plt.savefig('./results/turning_angles/T_eff_smooth.png')
if show_verb:
    plt.show()
else:
    plt.close()